### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
sdc_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
sdc_df.dtypes
sdc_df.count()
sdc_df.columns
sdc_df.head()
school_data_df = school_data.set_index('school_name')
school_data_df.drop(['School ID', 'size','budget'], axis=1)

,type
school_name,
Huang High School,District
Figueroa High School,District
Shelton High School,Charter
Hernandez High School,District
Griffin High School,Charter
Wilson High School,Charter
Cabrera High School,Charter
Bailey High School,District
Holden High School,Charter


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [14]:
# Calculate summary metrics
school_count = len(sdc_df['school_name'].unique())
student_count = len(sdc_df['student_name'])
total_budget = sdc_df['budget'].unique().sum()
ave_math_score = sdc_df['math_score'].mean()
ave_reading_score = sdc_df['reading_score'].mean()
math_pass_score_count = len(sdc_df[sdc_df['math_score']>=70])
math_pass_percent = (math_pass_score_count/student_count) * 100
reading_pass_score_count = len(sdc_df[sdc_df['reading_score']>=70])
reading_pass_percent = (reading_pass_score_count/student_count) * 100
overall_pass_count = len(sdc_df[(sdc_df['math_score']>=70) & (sdc_df['reading_score']>=70)])
overall_pass_percent = (overall_pass_count/student_count)*100

dist_sum_df = pd.DataFrame({
    'Number of Schools':[school_count], 
    'Number of Students':[student_count], 
    'Total Budget':[total_budget], 
    'Average Math Score':[ave_math_score], 
    'Average English Score':[ave_reading_score],
    '% Passing Math':[math_pass_percent],
    '% Passing English':[reading_pass_percent],
    '% Overall Passing': [overall_pass_percent]   
})

#dist_sum_df['Total Budget'] = dist_sum_df['Total Budget'].map('${:.2f}'.format)
dist_sum_df['Total Budget'] = dist_sum_df['Total Budget'].map("{:,}".format)
dist_sum_df

,Number of Schools,Number of Students,Total Budget,Average Math Score,Average English Score,% Passing Math,% Passing English,% Overall Passing
0,15,39170,"24,649,428",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [16]:
total_students = sdc_df['school_name'].value_counts()
total_students_df = pd.DataFrame(total_students)
total_students_df.columns = ['total_students']

grouped_schools = sdc_df.groupby(['school_name'])
school_type = grouped_schools['type'].unique()
school_type_df = pd.DataFrame(school_type)


school_budget = grouped_schools['budget'].mean()
school_budget_df = pd.DataFrame(school_budget)
school_budget_df['budget'] = school_budget_df.loc[:, 'budget'].astype(float)

budget_per_student = school_budget/total_students
budget_per_student_df = pd.DataFrame(budget_per_student)
budget_per_student_df.columns = ['Per Student Budget']

math_average = grouped_schools['math_score'].mean()
math_average_df = pd.DataFrame(math_average)

reading_average = grouped_schools['reading_score'].mean()
reading_average_df = pd.DataFrame(reading_average)


# 1. Here I find the number of students that passed math
sdc_math_pass = sdc_df[sdc_df['math_score']>=70]
math_pass_count_df = sdc_math_pass[['school_name','student_name']].groupby(['school_name']).count()
math_pass_count_df.columns = ['math_pass_count']

# 2. Here I find the number of students that passed english
sdc_reading_pass = sdc_df[sdc_df['reading_score']>=70]
reading_pass_count_df = sdc_reading_pass[['school_name','student_name']].groupby(['school_name']).count()
reading_pass_count_df.columns = ['reading_pass_count']

# 3. Here I find the number of students that passed both math and english
sdc_overall_pass = sdc_df[(sdc_df['math_score']>=70) & (sdc_df['reading_score']>=70)]
overall_pass_count_df = sdc_overall_pass[['school_name','student_name']].groupby(['school_name']).count()
overall_pass_count_df.columns = ['overall_pass_count']

# 4. I now combine the three with the school data and total students dataframes and calculate the percentages of passes
combined_df = pd.concat([total_students_df, math_pass_count_df, reading_pass_count_df, overall_pass_count_df, school_data_df], axis=1, join='inner')
combined_df = combined_df.assign(percent_passing_math = lambda x: (x['math_pass_count'] /x['total_students'] * 100))
combined_df = combined_df.assign(percent_passing_reading = lambda x: (x['reading_pass_count'] /x['total_students'] * 100))
combined_df = combined_df.assign(percent_passing_overall = lambda x: (x['overall_pass_count'] /x['total_students'] * 100))
reduced_df = combined_df.drop(['size','math_pass_count','reading_pass_count','overall_pass_count','School ID'], axis=1)

sch_sum_df = pd.concat([reduced_df, budget_per_student_df, math_average_df, reading_average_df], axis=1, join='inner')
sch_sum_df

# Rename columns to match those in solution
school_sum_df = sch_sum_df.rename(columns={'total_students': 'Total Students', 
                                  'type': 'School Type',
                                  'budget': 'Total School Budget',
                                  'percent_passing_math': '% Passing Math',
                                  'percent_passing_reading': '% Passing Reading',
                                  'percent_passing_overall':'% Overall Passing',
                                  'math_score':'Average Math Score',
                                  'reading_score':'Average Reading Score'})

# Rearranging columns to get the final table
school_summary_df = school_sum_df[['School Type','Total Students','Total School Budget','Per Student Budget',
                                   'Average Math Score','Average Reading Score','% Passing Math',
                                   '% Passing Reading','% Overall Passing']]

school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Wright High School,Charter,1800,1049400,583.0,83.682222,83.955000,93.333333,96.611111,90.333333


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [17]:
top_five_df = school_summary_df.sort_values('% Overall Passing', ascending=False)
top_five_df.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [18]:
Bottom_five_df = school_summary_df.sort_values('% Overall Passing')
Bottom_five_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [19]:
#school_grade_group = sdc_df.groupby(['school_name', 'grade'])
#math_score_by_grade = school_grade_group['math_score'].mean()
#math_score_by_grade_df = pd.DataFrame(math_score_by_grade)
#math_score_by_grade_df
#math_score_by_grade_df.unstack('grade')
#math_score_by_grade_df.columns = ['school_name',]



math_score_by_grade = sdc_df[['grade','school_name','math_score']].groupby(['grade','school_name']).mean()
math_score_by_grade

math_score_by_grade_df = pd.DataFrame(math_score_by_grade)
math_score_by_grade_df
math_score_by_grade.unstack('grade')

#math_by_grade_df = math_score_by_grade_df.set_index(['school_name'] + ['9th', '10th', '11th', '12th']).unstack('grade')

#new_df = df.set_index(['name'] + metas).unstack('name'))

#math_score_by_grade_df=math_score_by_grade_df['9th','10th','11th','12th']

math_score                                 
grade                       10th       11th       12th        9th
school_name                                                      
Bailey High School     76.996772  77.515588  76.492218  77.083676
Cabrera High School    83.154506  82.765560  83.277487  83.094697
Figueroa High School   76.539974  76.884344  77.151369  76.403037
Ford High School       77.672316  76.918058  76.179963  77.361345
Griffin High School    84.229064  83.842105  83.356164  82.044010
Hernandez High School  77.337408  77.136029  77.186567  77.438495
Holden High School     83.429825  85.000000  82.855422  83.787402
Huang High School      75.908735  76.446602  77.225641  77.027251
Johnson High School    76.691117  77.491653  76.863248  77.187857
Pena High School       83.372000  84.328125  84.121547  83.625455
Rodriguez High School  76.612500  76.395626  77.690748  76.859966
Shelton High School    82.917411  83.383495  83.778976  83.420755
Thomas High School     83.087886  83.498795  83.497041  83.590022
Wilson High School     83.724422  83.195326  83.035794  83.085578
Wright High School     84.010288  83.836782  83.644986  83.264706

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [20]:
reading_score_by_grade = sdc_df[['grade','school_name','reading_score']].groupby(['school_name', 'grade']).mean()
reading_score_by_grade_df = pd.DataFrame(reading_score_by_grade)
reading_score_by_grade_df

reading_score_by_grade_df.unstack('grade')
#reading_score_by_grade_df.columns.droplevel(0)

reading_score                                 
grade                          10th       11th       12th        9th
school_name                                                         
Bailey High School        80.907183  80.945643  80.912451  81.303155
Cabrera High School       84.253219  83.788382  84.287958  83.676136
Figueroa High School      81.408912  80.640339  81.384863  81.198598
Ford High School          81.262712  80.403642  80.662338  80.632653
Griffin High School       83.706897  84.288089  84.013699  83.369193
Hernandez High School     80.660147  81.396140  80.857143  80.866860
Holden High School        83.324561  83.815534  84.698795  83.677165
Huang High School         81.512386  81.417476  80.305983  81.290284
Johnson High School       80.773431  80.616027  81.227564  81.260714
Pena High School          83.612000  84.335938  84.591160  83.807273
Rodriguez High School     80.629808  80.864811  80.376426  80.993127
Shelton High School       83.441964  84.373786  82.781671  84.122642
Thomas High School        84.254157  83.585542  83.831361  83.728850
Wilson High School        84.021452  83.764608  84.317673  83.939778
Wright High School        83.812757  84.156322  84.073171  83.833333

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [21]:
#school budget min & max values
print(school_summary_df ['Per Student Budget'].min())
print(school_summary_df ['Per Student Budget'].max())

# Create bins in which to place the different categories based on school spending
bins = [0, 585, 630, 645, 675]

#create labels for the bins
funding_brackets = ['Below 585', '585-630', '630-645', '545-675']

#create table with bins
school_summary_df['Budget Group'] = pd.cut(school_summary_df['Per Student Budget'], bins, labels = funding_brackets)
school_summary_df

budget_group = school_summary_df.groupby('Budget Group')
budget_group[['Average Math Score', 'Average Reading Score', '% Passing Math',  '% Passing Reading', '% Overall Passing']].mean()


578.0
655.0


C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Budget Group,,,,,
Below 585,83.455399,83.933814,93.460096,96.610877,90.369459
585-630,81.899826,83.155286,87.133538,92.718205,81.418596
630-645,78.518855,81.624473,73.484209,84.391793,62.857656
545-675,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

* Perform the same operations as above, based on school size.

In [22]:
#school size min & max values
print(school_summary_df ['Total Students'].min())
print(school_summary_df ['Total Students'].max())

# Create bins in which to place the different categories based on total no of students in each shool
bins = [0, 1000, 2000, 5000]

#create labels for the bins
school_size_brackets = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

#create table with bins
school_summary_df['School Size'] = pd.cut(school_summary_df['Total Students'], bins, labels = school_size_brackets)
school_summary_df

#Create groups with the school size in first column
school_size_group = school_summary_df.groupby('School Size')
school_size_group[['Average Math Score', 'Average Reading Score', '% Passing Math',  '% Passing Reading', '% Overall Passing']].mean()


427
4976


C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [23]:
# Calculating averages for district and charter Schools
ave_math_score = school_summary_df[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing', 'School Type']].groupby(['School Type']).mean()
ave_math_score

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
